In [1]:
cd C:\Users\Nicole\Desktop\31008DMP\dataset

C:\Users\Nicole\Desktop\31008DMP\dataset


In [2]:
import random
import numpy as np
import pandas as pd
import ast
import re
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
import string
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data_all = pd.read_csv('data.csv')
data_w_genres = pd.read_csv('data_w_genres.csv')
data_b_genres = pd.read_csv('data_by_genres.csv')
data_b_artist = pd.read_csv('data_by_artist.csv')
data_b_year = pd.read_csv('data_by_year.csv')

In [4]:
[data_all.shape, data_w_genres.shape, data_b_genres.shape, data_b_artist.shape, data_b_year.shape]

[(174389, 19), (32539, 16), (3232, 14), (32539, 15), (102, 14)]

In [5]:
data_all.columns

Index(['acousticness', 'artists', 'danceability', 'duration_ms', 'energy',
       'explicit', 'id', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'name', 'popularity', 'release_date', 'speechiness', 'tempo',
       'valence', 'year'],
      dtype='object')

In [5]:
data_all.head(20)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.2240,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.5170,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.1860,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.7980,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.7070,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920
5,0.996000,['Mamie Smith & Her Jazz Hounds'],0.424,198627,0.2450,0,3HnrHGLE9u2MjHtdobfWl9,0.799000,5,0.2350,-11.470,1,Crazy Blues - 78rpm Version,9,1920,0.0397,103.870,0.4770,1920
6,0.992000,['Mamie Smith'],0.782,195200,0.0573,0,5DlCyqLyX2AOVDTjjkDZ8x,0.000002,5,0.1760,-12.453,1,Don't You Advertise Your Man,5,1920,0.0592,85.652,0.4870,1920
7,0.996000,['Mamie Smith & Her Jazz Hounds'],0.474,186173,0.2390,0,02FzJbHtqElixxCmrpSCUa,0.186000,9,0.1950,-9.712,1,Arkansas Blues,0,1920,0.0289,78.784,0.3660,1920
8,0.996000,['Francisco Canaro'],0.469,146840,0.2380,0,02i59gYdjlhBmbbWhf8YuK,0.960000,8,0.1490,-18.717,1,La Chacarera - Remasterizado,0,1920-07-08,0.0741,130.060,0.6210,1920
9,0.006820,['Meetya'],0.571,476304,0.7530,0,06NUxS2XL3efRh0bloxkHm,0.873000,8,0.0920,-6.943,1,Broken Puppet - Original Mix,0,1920-01-01,0.0446,126.993,0.1190,1920


Number of Artists

In [8]:
data_all['artist_list'] = data_all.apply(lambda row: ast.literal_eval(row['artists']) , axis=1)
data_all['num_artists'] = data_all.apply(lambda row: len(row['artist_list']) , axis=1)

In [9]:
data_all['num_artists'].value_counts()

1     137514
2      24045
3       8075
4       2699
5        959
6        413
7        214
8        131
9        126
10        53
12        46
11        34
13        23
20        17
15        13
14         9
27         6
16         6
19         3
17         1
18         1
28         1
Name: num_artists, dtype: int64

In [13]:
num_artists_cond = [data_all['num_artists'] == 1,
                    data_all['num_artists'] == 2,
                    data_all['num_artists'] == 3,
                    data_all['num_artists'] >= 4]
num_artist_group = ['numart_1', 'numart_2', 'numart_3', 'numart_4_above']
data_all['num_artists_grp'] = np.select(num_artists_cond, num_artist_group, default='missing')
data_all['num_artists_grp'].value_counts()

numart_1          137514
numart_2           24045
numart_3            8075
numart_4_above      4755
Name: num_artists_grp, dtype: int64

In [14]:
data_all = data_all.drop(['num_artists'], axis=1)

Song name: any non-English?

In [15]:
new_name = [nm.translate(str.maketrans('', '', string.punctuation)) for nm in data_all['name']]
len(new_name)

174389

In [16]:
# I tried to loop through the name word by word but the detection didn't work well and it runs foever
# So now I'm trying to loop just by the name

nonen_flag = []
for item in new_name:
    try:
        if detect(item) != 'en':
            nonen_flag.append(1)
        else:
            nonen_flag.append(0)
                
    except LangDetectException: # Pass if there is a line break
        #print('line break')
        nonen_flag.append(0)
        #break

In [17]:
[len(nonen_flag), sum(nonen_flag), len(nonen_flag)-sum(nonen_flag)]

[174389, 82334, 92055]

In [18]:
#print(nonen_flag[0:10])
#print(new_name[0:10])

In [19]:
data_all['songname_noneng'] = nonen_flag

Num Words in Song Name

In [20]:
num_words_songname = [len(re.findall(r'\w+', nm))  for nm in data_all['name']]
num_words_songname[0:10]

[6, 6, 2, 8, 1, 4, 6, 2, 3, 4]

In [21]:
data_all['num_words_in_songname'] = num_words_songname

Group years to periods

In [22]:
data_all['period'] = ''
data_all.loc[data_all.year < 1950, 'period'] = '1920_to_1950'
data_all.loc[(data_all.year >= 1950) & (data_all.year < 1980), 'period'] = '1950_to_1980'
data_all.loc[(data_all.year >= 1980) & (data_all.year < 2000), 'period'] = '1980_to_2000'
data_all.loc[(data_all.year >= 2000) & (data_all.year < 2010), 'period'] = '2000_to_2010'
data_all.loc[data_all.year >= 2010, 'period'] = '2010_to_2020'
data_all['period'].value_counts()

1950_to_1980    59728
1980_to_2000    40402
1920_to_1950    31888
2010_to_2020    28589
2000_to_2010    13782
Name: period, dtype: int64

Expand based on artist

In [39]:
# test: df.explode
#df = pd.DataFrame({'A': [[1, 2, 3], 'foo', [], [3, 4]], 'B': [1,2,3,4]})
#df.explode('A')

In [23]:
print(data_all.shape)
data_all.head(5)

(174389, 24)


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,...,release_date,speechiness,tempo,valence,year,artist_list,num_artists_grp,songname_noneng,num_words_in_songname,period
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,...,1920,0.0936,149.976,0.6340,1920,[Mamie Smith],numart_1,0,6,1920_to_1950
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,...,1920-01-05,0.0534,86.889,0.9500,1920,[Screamin' Jay Hawkins],numart_1,0,6,1920_to_1950
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,...,1920,0.1740,97.600,0.6890,1920,[Mamie Smith],numart_1,1,2,1920_to_1950
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,...,1920-01-01,0.0425,127.997,0.0422,1920,[Oscar Velazquez],numart_1,1,8,1920_to_1950
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,...,1920-10-01,0.0768,122.076,0.2990,1920,[Mixe],numart_1,1,1,1920_to_1950


In [24]:
data_expand = data_all.explode('artist_list')
data_expand.shape

(234372, 24)

In [25]:
data_all[data_all['id']=='14YYUaft0fQCYn7IqL4FYV']

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,...,release_date,speechiness,tempo,valence,year,artist_list,num_artists_grp,songname_noneng,num_words_in_songname,period
51,0.365,"['Gustavo Obligado', 'Ariel Flores', 'Sam Nacht']",0.565,128500,0.255,1,14YYUaft0fQCYn7IqL4FYV,0.000287,10,0.321,...,1920,0.048,94.519,0.725,1920,"[Gustavo Obligado, Ariel Flores, Sam Nacht]",numart_3,0,8,1920_to_1950


In [26]:
data_expand[data_expand['id']=='14YYUaft0fQCYn7IqL4FYV']

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,...,release_date,speechiness,tempo,valence,year,artist_list,num_artists_grp,songname_noneng,num_words_in_songname,period
51,0.365,"['Gustavo Obligado', 'Ariel Flores', 'Sam Nacht']",0.565,128500,0.255,1,14YYUaft0fQCYn7IqL4FYV,0.000287,10,0.321,...,1920,0.048,94.519,0.725,1920,Gustavo Obligado,numart_3,0,8,1920_to_1950
51,0.365,"['Gustavo Obligado', 'Ariel Flores', 'Sam Nacht']",0.565,128500,0.255,1,14YYUaft0fQCYn7IqL4FYV,0.000287,10,0.321,...,1920,0.048,94.519,0.725,1920,Ariel Flores,numart_3,0,8,1920_to_1950
51,0.365,"['Gustavo Obligado', 'Ariel Flores', 'Sam Nacht']",0.565,128500,0.255,1,14YYUaft0fQCYn7IqL4FYV,0.000287,10,0.321,...,1920,0.048,94.519,0.725,1920,Sam Nacht,numart_3,0,8,1920_to_1950


In [27]:
data_expand.columns

Index(['acousticness', 'artists', 'danceability', 'duration_ms', 'energy',
       'explicit', 'id', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'name', 'popularity', 'release_date', 'speechiness', 'tempo',
       'valence', 'year', 'artist_list', 'num_artists_grp', 'songname_noneng',
       'num_words_in_songname', 'period'],
      dtype='object')

Create unique key of dataset

In [28]:
artist_song_id = data_expand.apply(lambda row: row['artist_list'].replace(" ", "_").lower() + '_' + row['id'], axis=1)
artist_song_id

0                   mamie_smith_0cS0A1fUEUd1EW3FcF8AEI
1         screamin'_jay_hawkins_0hbkKFIJm7Z05H8Zl9w30f
2                   mamie_smith_11m7laMUgmOKqI3oYzuhne
3               oscar_velazquez_19Lc5SfJJ5O1oaxY0fpwfh
4                          mixe_2hJjbsLCytGsnAHfdsLejp
                              ...                     
174384                   tony_t_46LhBf6TvYjZU2SMvGZAbn
174385             alessia_cara_7tue2Wemjd0FZzRtDrQFZd
174386                roger_fly_48Qj61hOdYmUCFJbpQ29Ob
174387             taylor_swift_1gcyHQpBQ1lfXGdhZmWrHP
174388                roger_fly_57tgYkWQTNHVFEt6xDKKZj
Length: 234372, dtype: object

In [29]:
data_expand['artist_song_id'] = artist_song_id

Attach year info

In [30]:
data_b_year.columns = [str(col) + '_year' for col in data_b_year.columns]

In [31]:
print(len(data_expand['year'].unique()))
print(data_b_year['year_year'].value_counts().sum())
print(data_b_year.shape)

102
102
(102, 14)


In [32]:
data_b_year.columns

Index(['year_year', 'acousticness_year', 'danceability_year',
       'duration_ms_year', 'energy_year', 'instrumentalness_year',
       'liveness_year', 'loudness_year', 'speechiness_year', 'tempo_year',
       'valence_year', 'popularity_year', 'key_year', 'mode_year'],
      dtype='object')

In [33]:
data_new = data_expand.merge(data_b_year, how='left', left_on='year', right_on='year_year')
data_new.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,...,energy_year,instrumentalness_year,liveness_year,loudness_year,speechiness_year,tempo_year,valence_year,popularity_year,key_year,mode_year
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,...,0.4187,0.354219,0.216049,-12.65402,0.082984,113.2269,0.49821,0.610315,2,1
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,...,0.4187,0.354219,0.216049,-12.65402,0.082984,113.2269,0.49821,0.610315,2,1
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,...,0.4187,0.354219,0.216049,-12.65402,0.082984,113.2269,0.49821,0.610315,2,1
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,...,0.4187,0.354219,0.216049,-12.65402,0.082984,113.2269,0.49821,0.610315,2,1
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,...,0.4187,0.354219,0.216049,-12.65402,0.082984,113.2269,0.49821,0.610315,2,1


Attach artist info

In [34]:
data_b_artist.columns = [str(col) + '_artist' for col in data_b_artist.columns]

In [35]:
data_b_artist.head(2)

,artists_artist,acousticness_artist,danceability_artist,duration_ms_artist,energy_artist,instrumentalness_artist,liveness_artist,loudness_artist,speechiness_artist,tempo_artist,valence_artist,popularity_artist,key_artist,mode_artist,count_artist
0,"""Cats"" 1981 Original London Cast",0.598500,0.470100,267072.0,0.376203,0.010261,0.283050,-14.4343,0.209150,114.128800,0.358320,38.200000,5,1,10
1,"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.0,0.406808,0.081158,0.315215,-10.6900,0.176212,103.044154,0.268865,31.538462,5,1,26


In [36]:
data_b_artist['artists_artist'][18059]

'Mamie Smith'

In [37]:
data_expand['artist_list'][0]

'Mamie Smith'

In [38]:
#artist_with_info = [nm for nm in data_expand['artist_list'].tolist() if nm in data_b_artist['artists_artist'].tolist()]
#len(artist_with_info) #234360

In [39]:
data_new2 = data_new.merge(data_b_artist, how='left', left_on='artist_list', right_on='artists_artist')
data_new2.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,...,instrumentalness_artist,liveness_artist,loudness_artist,speechiness_artist,tempo_artist,valence_artist,popularity_artist,key_artist,mode_artist,count_artist
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,...,0.000288,0.372625,-12.800313,0.121438,113.009938,0.641062,3.1875,0.0,1.0,16.0
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,...,0.033991,0.267488,-10.309187,0.071781,124.976125,0.699437,19.4375,5.0,1.0,32.0
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,...,0.000288,0.372625,-12.800313,0.121438,113.009938,0.641062,3.1875,0.0,1.0,16.0
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,...,0.856667,0.086300,-7.745667,0.048400,128.659667,0.120400,11.0000,2.0,1.0,6.0
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,...,0.000246,0.402000,-6.036000,0.076800,122.076000,0.299000,2.0000,10.0,0.0,2.0


Fill missing with median

In [40]:
def missing_imputation(input_df):
    
    col_list = [str(col) for col in input_df.columns.tolist()]
    
    for col in col_list:
        if input_df[col].dtype == 'object':
            fill_value = 'Missing'
            
        elif input_df[col].dtype == 'float' or input_df[col].dtype == 'int64':
            fill_value = input_df[col].median()
            
        input_df[col] = input_df[col].fillna(fill_value)
    return input_df

In [93]:
data_final = missing_imputation(data_new2)
data_final.head(5)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,...,liveness_artist,loudness_artist,speechiness_artist,tempo_artist,valence_artist,popularity_artist,key_artist,mode_artist,count_artist,popularity_target
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,...,0.372625,-12.800313,0.121438,113.009938,0.641062,3.1875,0.0,1.0,16.0,2
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,...,0.267488,-10.309187,0.071781,124.976125,0.699437,19.4375,5.0,1.0,32.0,2
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,...,0.372625,-12.800313,0.121438,113.009938,0.641062,3.1875,0.0,1.0,16.0,2
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,...,0.086300,-7.745667,0.048400,128.659667,0.120400,11.0000,2.0,1.0,6.0,3
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,...,0.402000,-6.036000,0.076800,122.076000,0.299000,2.0000,10.0,0.0,2.0,2


Group Target Popularity

In [94]:
print(sorted(data_final['popularity'].unique()))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 100]


In [95]:
popularity_cond = [data_final['popularity'] <= 0,
                   data_final['popularity'] <= 15,
                   data_final['popularity'] <= 30,
                   data_final['popularity'] <= 50,
                   data_final['popularity'] <= 100]
popularity_group = [1, 2, 3, 4, 5]
popularity_target = np.select(popularity_cond, popularity_group, default='missing')
#pd.Series(popularity_target).value_counts().sort_index()
data_final['popularity_target'] = popularity_target
data_final['popularity_target'].value_counts()

1    64041
4    55916
2    40608
3    40160
5    33647
Name: popularity_target, dtype: int64

List of useless columns

In [96]:
print(data_final.shape)
data_final.columns

(234372, 55)


Index(['acousticness', 'artists', 'danceability', 'duration_ms', 'energy',
       'explicit', 'id', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'name', 'popularity', 'release_date', 'speechiness', 'tempo',
       'valence', 'year', 'artist_list', 'num_artists_grp', 'songname_noneng',
       'num_words_in_songname', 'period', 'artist_song_id', 'year_year',
       'acousticness_year', 'danceability_year', 'duration_ms_year',
       'energy_year', 'instrumentalness_year', 'liveness_year',
       'loudness_year', 'speechiness_year', 'tempo_year', 'valence_year',
       'popularity_year', 'key_year', 'mode_year', 'artists_artist',
       'acousticness_artist', 'danceability_artist', 'duration_ms_artist',
       'energy_artist', 'instrumentalness_artist', 'liveness_artist',
       'loudness_artist', 'speechiness_artist', 'tempo_artist',
       'valence_artist', 'popularity_artist', 'key_artist', 'mode_artist',
       'count_artist', 'popularity_target'],
      dtype='ob

In [97]:
#drop useless columns
col_to_drop = ['release_date', 'year_year', 'popularity_year', 'artists_artist', 'popularity_artist','popularity']
data_final = data_final.drop(col_to_drop, axis=1)

#rename the artist name list
data_final = data_final.rename(columns={"artist_list": "artist_name"})

#df info
print(data_final.shape)
data_final.columns

(234372, 49)


Index(['acousticness', 'artists', 'danceability', 'duration_ms', 'energy',
       'explicit', 'id', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'name', 'speechiness', 'tempo', 'valence', 'year',
       'artist_name', 'num_artists_grp', 'songname_noneng',
       'num_words_in_songname', 'period', 'artist_song_id',
       'acousticness_year', 'danceability_year', 'duration_ms_year',
       'energy_year', 'instrumentalness_year', 'liveness_year',
       'loudness_year', 'speechiness_year', 'tempo_year', 'valence_year',
       'key_year', 'mode_year', 'acousticness_artist', 'danceability_artist',
       'duration_ms_artist', 'energy_artist', 'instrumentalness_artist',
       'liveness_artist', 'loudness_artist', 'speechiness_artist',
       'tempo_artist', 'valence_artist', 'key_artist', 'mode_artist',
       'count_artist', 'popularity_target'],
      dtype='object')

In [98]:
#mark column roles
col_to_exclude = ['artist_song_id', 'artists', 'artist_name', 'id', 'name', 'year']
col_to_dummy = ['period', 'num_artists_grp']
target_var = 'popularity_target'

In [99]:
'num_artists_grp' in data_final.columns

True

Output dataset

In [100]:
data_final.to_csv('data_final_popularity.csv', index=False)  